In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import statsmodels.api as sm 
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import pprint as pp
import itertools
from collections import ChainMap
from sklearn.model_selection import cross_validate
import datetime
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
df = pd.read_hdf('Data/preprocessed/df_ml_hdf.h5', key='match')
df.dropna(inplace=True)

def preselect_non_redundant_features(df, target, corr_threshold):
    corr = df.corr()
    corr[target] = abs(corr[target])
    corr = corr.sort_values(target, ascending=False)

    #prepare lower trainagular matrix (with deleted diagonal)
    low_tri_no_diag = np.tril(np.ones(corr.shape).astype(np.bool), k=-1)
    corr = corr.where(low_tri_no_diag)
    #drop tagret variable form rows and columns
    corr = corr.drop(corr.columns[0], axis=1).drop(corr.index[0], axis=0)

    corr['MaxCorr'] = corr.abs().max(axis = 1)

    pre_selected_features = corr[corr['MaxCorr'] < corr_threshold].index 
    return pre_selected_features
    
def get_non_redundant_df(df, target, corr_threshold):
    features = preselect_non_redundant_features(df, target, corr_threshold)
    output = df.loc[:,features]
    return output


def feature_based_train_test_split(df, target, splitting_feature, test_split_value, 
                                   splitting_feature_as_index=True):
    if splitting_feature_as_index == True:
        X_train = df[~df.index.isin([test_split_value], level = splitting_feature)]
        X_test = df[df.index.isin([test_split_value], level = splitting_feature)]
                                  
        y_train = df[~df.index.isin([test_split_value], level = splitting_feature)].reset_index()[target].values
        y_test = df[df.index.isin([test_split_value], level = splitting_feature)].reset_index()[target].values
                   
    else:
        X_train = df[~df[splitting_feature]==test_split_value]
        X_test = df[df[splitting_feature]==test_split_value]
        
        y_train = df[~df[splitting_feature]==test_split_value].reset_index()[target].values
        y_test = df[df[splitting_feature]==test_split_value].reset_index()[target].values
        
    return X_train, y_train, X_test, y_test

  
#df = get_non_redundant_df(df, 'PTS_Result', 0.5) 
       
X_train, y_train, X_test, y_test = feature_based_train_test_split(df, 'Won_Result', 'season', '2019-20')    
    

In [ ]:
df.columns = df.columns.to_flat_index()
df.head()

In [ ]:
df.to_csv('Data/preprocessed/non_redundant_df.csv')

In [ ]:
###Clusterize


X_train_sc, X_test_sc = StandardScaler().fit_transform(X_train), StandardScaler().fit_transform(X_test)
# Create a PCA instance: pca
pca = PCA(n_components=10)
principalComponents_train = pca.fit_transform(X_train_sc)
principalComponents_test = pca.fit_transform(X_test_sc)
# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)

# Save components to a DataFrame
PCA_components_train = pd.DataFrame(principalComponents_train)
PCA_components_test = pd.DataFrame(principalComponents_test)

In [ ]:
plt.scatter(PCA_components_train[0], PCA_components_train[1], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Train Data')

In [ ]:
plt.scatter(PCA_components_test[0], PCA_components_test[1], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Test Data')

In [ ]:
ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(PCA_components.iloc[:,:3])

    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()